In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adamax
from keras.wrappers.scikit_learn import KerasRegressor

In [2]:
import pandas as pd
import numpy as np

# 1035, 258
train = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/spmm-latency-traintest/train-test-csv/nonsquare-train-1035-from-spmm-contain-todense-over-3s-1293.csv')
test = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/spmm-latency-traintest/train-test-csv/nonsquare-test-258-from-spmm-contain-todense-over-3s-1293.csv')

In [3]:
# Train
X_train = train[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc']] 
y_train = train['sp_smdm']

# Test
X_test = test[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc']] 
y_test = test['sp_smdm']

In [4]:
# 데이터 표준화(Standardization), 회귀 문제에선 MinMaxScaler가 좋음
from sklearn.preprocessing import MinMaxScaler

# 변형 객체 생성
minmax_scaler = MinMaxScaler()

# 훈련데이터의 모수 분포 저장
minmax_scaler.fit(X_train)

# 훈련 데이터 스케일링
X_train = minmax_scaler.transform(X_train)

# 테스트 데이터의 스케일링
X_test = minmax_scaler.transform(X_test)

In [5]:
# RMSE(train metric)
from keras import backend as K

def rmse(y_true, y_pred):
    rmse = K.sqrt(K.mean(K.square(y_pred - y_true))) 
    return rmse

In [6]:
# def mean_absolute_percentage_error(y_test, y_pred):
#     y_test, y_pred = np.array(y_test), np.array(y_pred)
#     return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

In [7]:
from sklearn.metrics import mean_squared_error

def mape_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

def rmse_error(y_true, y_pred):
    rmse = np.sqrt(np.mean(np.square(y_pred - y_true))) 
    return rmse

### 모델 생성

In [8]:
# hyperparameter tuning 대상 모델 정의
def create_model(dense_nparams, dense_layer_sizes , input_optimizer, input_kernel_initializer, input_dropout, input_lr):

    model=Sequential()
    model.add(Dense(dense_nparams, activation="relu", input_shape=(X_train.shape[1],), kernel_initializer=input_kernel_initializer))  
    model.add(Dropout(input_dropout),)
    
    # dense_layer_sizes 만큼 layer 추가
    for layer_size in dense_layer_sizes:
        model.add(Dense(layer_size, activation='relu', kernel_initializer=input_kernel_initializer))
        model.add(Dropout(input_dropout), )
    
    model.add(Dense(1))

    optimizer = input_optimizer(lr=input_lr)
    
    model.compile(optimizer = optimizer ,
                  loss='mape',
                  metrics=['mape',rmse])
    
    # 에포크가 끝날 때마다 점(.)을 출력해 훈련 진행 과정을 표시합니다
    class PrintDot(keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs):
            if epoch % 100 == 0: print('')
            #print("epoch : {}, logs : {}".format(epoch,logs))
            print('.', end='')

    # monitor는 어떤 매개변수를 볼 것인지 입니다.
    # patience 매개변수는 성능 향상을 체크할 에포크 횟수입니다
    # 지정된 에포크 횟수 동안 성능 향상이 없으면 자동으로 훈련이 멈춥니다.
    early_stop = keras.callbacks.EarlyStopping(monitor='val_mape', patience=50)

    EPOCHS = 1000

    # 훈련 정확도와 검증 정확도 출력
    # 에포크마다 훈련 상태를 점검하기 위해 EarlyStopping 콜백(callback)을 사용합니다.
    history = model.fit(X_train, 
                        y_train,
                        epochs=EPOCHS, 
                        validation_split = 0.1, 
                        verbose =0, 
                        callbacks=[early_stop, PrintDot()])
    # 예측값 출력
    y_pred = model.predict(X_test).reshape(-1,)
    
    # 오차율 계산
    mape = mape_error(y_test,y_pred)
    
    return mape

### hyperparameter 정의

In [9]:
# hyperparameter dictionary 화
param_grid = {
    # dense_nparams : 초기 dense layer size
    'dense_nparams' : [1024],
    # dense_layer_sizes : 사용할 dense layer size 목록
    'dense_layer_sizes' : [(256,64,16,)],
    # input_optimizer = optimizer
    'input_optimizer' : [SGD, Adagrad, RMSprop, Adam, Adamax],
    # input_kernel_initializer : 가중치 초기화 방법
    'input_kernel_initializer' :  ['uniform', 'normal', 
                                'glorot_uniform', 'glorot_normal',
                                'he_uniform', 'he_normal' ],
    # input_dropout : dropout 비율
    'input_dropout' : [0, 0.1, 0.2, 0.3, 0.4, 0.5],
    # input_lr : learning_rate
    'input_lr' : [0.001, 0.01, 0.1, 0.2]
}

In [10]:
# # dense_nparams : 초기 dense layer size
# dense_nparams = [1024]
# # dense_layer_sizes : 사용할 dense layer size 목록
# dense_layer_sizes = [(256,64,16,)]
# # input_optimizer = optimizer
# input_optimizer = [SGD, Adagrad, RMSprop, Adam, Adamax]
# # input_kernel_initializer : 가중치 초기화 방법
# input_kernel_initializer =  ['uniform', 'normal', 
#                             'glorot_uniform', 'glorot_normal',
#                             'he_uniform', 'he_normal' ]
# # input_dropout : dropout 비율
# input_dropout = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
# # input_lr : learning_rate
# input_lr = [0.001, 0.01, 0.1, 0.2]

# # hyperparameter dictionary 화
# param_grid = dict(dense_nparams = dense_nparams,
#                 dense_layer_sizes = dense_layer_sizes,
#                 input_optimizer = input_optimizer,
#                 input_kernel_initializer = input_kernel_initializer,
#                 input_dropout = input_dropout,
#                 input_lr = input_lr)

### BO 정의 및 시작

In [11]:
#  bayesian-optimization 라이브러리의 BayesianOptimization 클래스 import
from bayes_opt import BayesianOptimization

# Bayesian optimization 객체 생성
# f : 탐색 대상 함수, pbounds : hyperparameter 집합
# verbose = 2 항상 출력, verbose = 1 최댓값일 때 출력, verbose = 0 출력 안함
# random_state : Bayesian Optimization 상의 랜덤성이 존재하는 부분을 통제 
bo=BayesianOptimization(f=create_model, pbounds=param_grid, verbose=2, random_state=1 )    

# 메소드를 이용해 최대화 과정 수행
# init_points :  초기 Random Search 갯수
# n_iter : 반복 횟수 (몇개의 입력값-함숫값 점들을 확인할지! 많을 수록 정확한 값을 얻을 수 있다.)
# acq : Acquisition Function들 중 Expected Improvement(EI) 를 사용
# xi : exploration 강도 (기본값은 0.0)
bo.maximize(init_points=2, n_iter=10, acq='ei', xi=0.01)

# ‘iter’는 반복 회차, ‘target’은 목적 함수의 값, 나머지는 입력값을 나타냅니다. 
# 현재 회차 이전까지 조사된 함숫값들과 비교하여, 현재 회차에 최댓값이 얻어진 경우, 
# bayesian-optimization 라이브러리는 이를 자동으로 다른 색 글자로 표시하는 것을 확인할 수 있습니다

# 찾은 파라미터 값 확인
print(bo.max)

ValueError: setting an array element with a sequence.

In [ ]:
# # cross_validation 정의
# kf = KFold(random_state=30,
#            # Fold 는 10개로 지정
#            n_splits=10,
#            shuffle=True
#           )

# # gridsearch 정의
# grid = GridSearchCV(estimator=regressor_model, 
#                     param_grid=param_grid, 
#                     # scoring : 검증셋의 성능을 무엇으로 측정할 것인지
#                     scoring = make_scorer(mean_absolute_percentage_error, greater_is_better=False),
#                     cv = kf,
#                     # n_jobs : 프로세스가 시스템의 모든 코어를 사용하도록    
#                     n_jobs=-1,
#                     # 모든 log 출력하도록
#                     verbose=3)

# # gridsearch 시작
# grid_result = grid.fit(X_train, y_train)

# # gridesearch 결과
# print("최고의 파라미터 :", grid_result.best_params_)
# print("최고 평균 정확도 : {}".format(grid_result.best_score_))